In [1]:
import torch
print("GPU available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


GPU available? False


In [1]:
!pip install torch torchvision scikit-learn matplotlib pandas


In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from PIL import Image


In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

train_df['filepath'] = train_df['id_code'].apply(lambda x: f"data/train_images/{x}.png")
test_df['filepath'] = test_df['id_code'].apply(lambda x: f"data/test_images/{x}.png")

train_df.head()


,id_code,diagnosis,filepath
0,000c1434d8d7,2,data/train_images/000c1434d8d7.png
1,001639a390f0,4,data/train_images/001639a390f0.png
2,0024cdab0c1e,1,data/train_images/0024cdab0c1e.png
3,002c21358ce6,0,data/train_images/002c21358ce6.png
4,005b95c28852,0,data/train_images/005b95c28852.png


In [4]:
train_df_split, val_df_split = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['diagnosis'],
    random_state=42
)


In [5]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std= [0.229, 0.224, 0.225])
])



In [6]:
class RetinopathyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img = Image.open(row['filepath']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(row['diagnosis'], dtype=torch.long)
        return img, label


In [7]:
batch_size = 16

train_ds = RetinopathyDataset(train_df_split, transform=data_transforms)
val_ds = RetinopathyDataset(val_df_split, transform=data_transforms)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5)
model = model.to(device)


In [9]:
class_weights = compute_class_weight('balanced',
                                     classes=np.unique(train_df['diagnosis']),
                                     y=train_df['diagnosis'])

weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [10]:
for name, param in model.named_parameters():
    if "layer2" in name or "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [11]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


In [12]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}")
    
    scheduler.step()


Epoch 0, Batch 0, Loss: 1.6903
Epoch 0, Batch 10, Loss: 1.7192
Epoch 0, Batch 20, Loss: 1.7261
Epoch 0, Batch 30, Loss: 1.5960
Epoch 0, Batch 40, Loss: 1.6742
Epoch 0, Batch 50, Loss: 1.3711
Epoch 0, Batch 60, Loss: 1.5401
Epoch 0, Batch 70, Loss: 1.4416
Epoch 0, Batch 80, Loss: 1.7228
Epoch 0, Batch 90, Loss: 1.3808
Epoch 0, Batch 100, Loss: 1.4534
Epoch 0, Batch 110, Loss: 1.3839
Epoch 0, Batch 120, Loss: 1.4745
Epoch 0, Batch 130, Loss: 1.4609
Epoch 0, Batch 140, Loss: 1.3181
Epoch 0, Batch 150, Loss: 1.3907
Epoch 0, Batch 160, Loss: 1.4253
Epoch 0, Batch 170, Loss: 1.4937
Epoch 0, Batch 180, Loss: 1.2909
Epoch 1, Batch 0, Loss: 1.4724
Epoch 1, Batch 10, Loss: 1.3333
Epoch 1, Batch 20, Loss: 1.6228
Epoch 1, Batch 30, Loss: 1.1856
Epoch 1, Batch 40, Loss: 2.0418
Epoch 1, Batch 50, Loss: 1.1089
Epoch 1, Batch 60, Loss: 1.2280
Epoch 1, Batch 70, Loss: 1.2579
Epoch 1, Batch 80, Loss: 1.5039
Epoch 1, Batch 90, Loss: 1.5523
Epoch 1, Batch 100, Loss: 1.3345
Epoch 1, Batch 110, Loss: 1.4999

In [14]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 76.53%


In [15]:
model  # your trained model



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
import os
os.makedirs('models', exist_ok=True)


In [17]:
import torch

torch.save(model.state_dict(), 'models/best_model.pth')
print("✅ Trained model saved as 'models/best_model.pth'")


✅ Trained model saved as 'models/best_model.pth'


In [10]:
!pip install efficientnet-pytorch


In [2]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Use EfficientNet‑B0 for faster training on CPU
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=5)
model = model.to(device)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\namra/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth


100%|█████████████████████████████████████████████████████████████████████████████| 20.4M/20.4M [00:19<00:00, 1.13MB/s]

Loaded pretrained weights for efficientnet-b0


In [3]:
from torchvision import transforms

data_transforms = transforms.Compose([
    transforms.Resize((260, 260)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Crop to 224x224 for faster training
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15, fill=0),

    # Added small brightness/contrast variation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [9]:
%pip install albumentations
%pip install albumentations[torch]


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install albumentations --user


  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl (38.9 MB)

   ---------------------------------------- 0/3 [opencv-python-headless]
   ---------------------------------------- 0/3 [opencv-python-headless]
   ---------------------------------------- 0/3 [opencv-python-headless]
   ------------- -------------------------- 1/3 [albucore]
   -------------------------- ------------- 2/3 [albumentations]
   ---------------------------------------- 3/3 [albumentations]

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

train_transforms = A.Compose([
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=0.3),
    A.RandomResizedCrop(224, 224, scale=(0.8,1.0), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, p=0.5),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2(),
])


ModuleNotFoundError: No module named 'albumentations'

In [14]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class_weights = compute_class_weight('balanced',
                                     classes=np.unique(train_df['diagnosis']),
                                     y=train_df['diagnosis'])
weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


In [15]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


In [16]:
train_df = pd.read_csv('data/train.csv')


In [17]:
train_df['filepath'] = train_df['id_code'].apply(lambda x: f"data/train_images/{x}.png")


In [18]:
test_df = pd.read_csv('data/test.csv')
test_df['filepath'] = test_df['id_code'].apply(lambda x: f"data/test_images/{x}.png")


In [19]:
import os

missing = train_df.loc[~train_df['filepath'].apply(os.path.exists), 'filepath']
print("Missing files:", len(missing))
if not missing.empty:
    print(missing.head())


Missing files: 0


In [20]:
from torchvision import transforms
import cv2
from PIL import Image
import numpy as np

def clahe_pil(img):
    img = np.array(img.convert('RGB'))[...,::-1]  # to OpenCV BGR
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab[...,0] = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(lab[...,0])
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return Image.fromarray(img[...,::-1])        # back to RGB PIL

data_transforms = transforms.Compose([
    transforms.Lambda(clahe_pil),
    transforms.Resize((260, 260)),
    transforms.RandomResizedCrop(260, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])


In [21]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch

class RetinopathyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img = Image.open(row['filepath']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(row['diagnosis'], dtype=torch.long)
        return img, label

# Split 80/20 for training + validation
train_size = int(0.8 * len(train_df))
val_size   = len(train_df) - train_size
train_df_subset, val_df_subset = random_split(train_df, [train_size, val_size],
                                              generator=torch.Generator().manual_seed(42))

# Build datasets
train_ds = RetinopathyDataset(train_df.loc[train_df_subset.indices], transform=data_transforms)
val_ds   = RetinopathyDataset(train_df.loc[val_df_subset.indices], transform=data_transforms)

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)


In [22]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['diagnosis']),
    y=train_df['diagnosis']
)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
num_epochs = 10
best_val = 0

for epoch in range(num_epochs):
    # Training
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # ⬅️ This line should be aligned here (no extra spaces before 'if')
        if batch_idx % 20 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, "
                  f"Batch {batch_idx}/{len(train_loader)}, "
                  f"Loss: {loss.item():.4f}")

    scheduler.step()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            _, preds = torch.max(model(images), 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    val_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} — Val Accuracy: {val_acc:.2f}%")

    if val_acc > best_val:
        best_val = val_acc
        torch.save(model.state_dict(), 'models/best_model.pth')
        print(f"✅ Saved best model at epoch {epoch+1}")
